In [ ]:
import numpy as np
import pandas as pd
import time

# 1. 시드 고정
np.random.seed(42)

# 2. 설정
n_users = 100
n_items = 500

# 3. 저장소
users = []
items = []
timestamps = []
## retrieve and reranking


# 모집단 미리 생성 (1 ~ 500)
all_item_ids = np.arange(1, n_items + 1)

# 4. 루프
for u in range(1, n_users + 1):
    # (1) 활동량 결정
    raw_count = int(np.random.pareto(a=2.0) * 40) + 10
    
    # [Fix] 활동량이 전체 아이템 수(500)를 넘지 않도록 제한(Clipping)
    count = min(raw_count, n_items)
    
    # (2) 아이템 선택 (이제 count는 절대 500을 넘지 않음)
    selected_items = np.random.choice(all_item_ids, size=count, replace=False)

    # (3) 시간 생성
    current_time = time.time()
    selected_times = current_time - np.random.randint(0, 30*24*3600, size=count)
    
    # 리스트 추가
    users.extend([u] * count)
    items.extend(selected_items)
    timestamps.extend(selected_times)

# 5. DataFrame 생성
train_ratings = pd.DataFrame({
    'user': users, 
    'item': items, 
    'time': timestamps
})

train_ratings['item'] = train_ratings['item'].astype(int)
train_ratings['time'] = train_ratings['time'].astype(int)

# 검증
print(f"Total Interactions: {len(train_ratings)}")
print(train_ratings['user'].value_counts().describe())
train_ratings.head()

Total Interactions: 4804
count    100.000000
mean      48.040000
std       70.358953
min       10.000000
25%       16.000000
50%       24.000000
75%       47.250000
max      500.000000
Name: count, dtype: float64


,user,item,time
0,1,269,1765583617
1,1,74,1766198875
2,1,290,1766279610
3,1,156,1765118147
4,1,105,1764364150


In [ ]:
import random
from tqdm import tqdm

# ---------------------------------------------------------
# Step 2. Candidate Generation (Simulation)
# ---------------------------------------------------------

# 1. 빠른 검색을 위한 'User Seen Dict' 생성 (Pre-processing)
# 데이터프레임의 groupby를 써도 되지만, 순수 파이썬 로직 훈련을 위해 for문 사용 추천
user_seen_dict = train_ratings.groupby('user')['item'].apply(set).to_dict()

# 2. 학습용 데이터(Train Instances) 생성
train_data = []

# 모든 유저에 대해 반복 (tqdm으로 진행도 표시)
for user_id in tqdm(user_seen_dict.keys(), desc="Generating Candidates"):
    
    # (A) Positive Instances (GT, Label 1)
    seen_items = user_seen_dict[user_id]
    for item_id in seen_items:
        train_data.append([user_id, item_id, 1])
    
    # (B) Negative Instances (Label 0)
    # 목표: (총 50개 - 본 개수) 만큼의 빈 자리를 Negative로 채우기
    target_neg_count = max(0, 50 - len(seen_items))
    
    neg_items = []
    
    # TODO 1: target_neg_count 만큼 Negative 아이템을 모으는 while 루프 작성
    # 조건: 모인 개수가 target_neg_count보다 작을 동안 반복
    # 행동 1: 1~500 사이 랜덤 아이템 하나 뽑기 (random.randint 사용)
    # 행동 2: 그 아이템이 seen_items에 없고, neg_items에도 없으면 추가
    
    while len(neg_items) < target_neg_count:
        # (User Code Here)
        
        
        
    
    # TODO 2: 수집된 neg_items를 train_data에 추가 (Label 0)
    for item_id in neg_items:
        # (User Code Here)
        

# 3. DataFrame 변환
df_train = pd.DataFrame(train_data, columns=['user_id', 'item_id', 'label'])

# 검증
print(f"\nTotal Train Samples: {len(df_train)}")
print(df_train['label'].value_counts())
df_train.head()